In [94]:
#Importamos las librerias que vamos a utilizar

import csv
import pandas as pd


In [95]:
#Ruta de los archivos CSV


movies = r'C:\Users\jotad\OneDrive\Documentos\Sebastian\DATA SCIENCE - PART TIME\PROYECTO INDIVIDUAL UNO\Dataset-20230613T023601Z-001\Dataset\movies_dataset.csv'
credits = r'C:\Users\jotad\OneDrive\Documentos\Sebastian\DATA SCIENCE - PART TIME\PROYECTO INDIVIDUAL UNO\Dataset-20230613T023601Z-001\Dataset\credits.csv'

df_movies = pd.read_csv(movies)
df_movies1 = pd.read_csv(movies)

C:\Users\jotad\AppData\Local\Temp\ipykernel_20068\674749560.py:7: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv(movies)
C:\Users\jotad\AppData\Local\Temp\ipykernel_20068\674749560.py:8: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies1 = pd.read_csv(movies)


In [96]:
#ESCRIBIMOS LAS FUNCIONES A UTILIZAR    

def desanidar_columnas(dataframe, columnas):
    desanidados = {}
    for columna in columnas:
        nueva_columna = columna + "_desanidada"
        desanidados[nueva_columna] = []
        for valor in dataframe[columna]:
            desanidados[nueva_columna].append(desanidar_valor(valor))
    return desanidados


def desanidar_valor(valor):
    if isinstance(valor, dict):
        return desanidar_diccionario(valor)
    elif isinstance(valor, list):
        return desanidar_lista(valor)
    else:
        return valor

def desanidar_diccionario(diccionario):
    desanidado = {}
    for clave, valor in diccionario.items():
        if isinstance(valor, dict):
            desanidado[clave] = desanidar_diccionario(valor)
        elif isinstance(valor, list):
            desanidado[clave] = desanidar_lista(valor)
        else:
            desanidado[clave] = valor
    return desanidado

def desanidar_lista(lista):
    desanidado = []
    for elemento in lista:
        if isinstance(elemento, dict):
            desanidado.append(desanidar_diccionario(elemento))
        elif isinstance(elemento, list):
            desanidado.append(desanidar_lista(elemento))
        else:
            desanidado.append(elemento)
    return desanidado

#Creamos una función para agregar las columnas desanidadas

def agregar_columnas_desanidadas(df, columnas_anidadas):
    # Creamos una copia del DataFrame original
    df_modificado = df.copy()

    # Recorremos las columnas desanidadas y agregamos una a la vez al DataFrame copiado 'df_modificado'
    for columna in columnas_anidadas:
        valores = df_modificado[columna]
        df_modificado[columna + "_desanidada"] = pd.Series(valores)

    # Devolver el DataFrame modificado
    return df_modificado

#Creamos una función para reemplazar los valores nulos de las columnas 'budget' y 'revenue' por el valor cero

def reemplazar_nulos_por_cero(dataframe, columnas):
    dataframe[columnas] = dataframe[columnas].fillna(0)
    return dataframe


#Creamos una función para eliminar los valores nulos de una columna

def eliminar_nulos(dataframe, columnas):
    dataframe[columnas] = dataframe[columnas].dropna()
    return dataframe


#Creamos una función para eliminar las columnas que no vamos a utilizar

def eliminar_columnas(dataframe, columnas):
    dataframe.drop(columns=columnas, inplace=True)
    return dataframe

#Creamos una función para calcular el retorno y convertir las columnas "revenue" y "budget" al tipo de dato numérico

def calcular_retorno(dataframe):
    dataframe["revenue"] = pd.to_numeric(dataframe["revenue"], errors="coerce")
    dataframe["budget"] = pd.to_numeric(dataframe["budget"], errors="coerce")
    dataframe["return"] = dataframe["revenue"].div(dataframe["budget"], fill_value=0)
    return dataframe

#Creamos una función para formatear las fechas
def formatear_fechas(dataframe, columna_fecha):
    # Convertir la columna de fecha al formato deseado (AAAA-mm-dd)
    dataframe[columna_fecha] = pd.to_datetime(dataframe[columna_fecha], errors='coerce').dt.strftime('%Y-%m-%d')
        
    return dataframe



#Creamos una función para transformar los datos y quitar el .0 de los valores de las columnas

def formatear_columnas(dataframe, *columnas):
    for columna in columnas:
        dataframe[columna] = dataframe[columna].fillna(0).astype(int).astype(str)
    return dataframe


#Creamos una función para reemplazar los valores nulos o faltantes por el texto 'Información no disponible'

def reemplazar_valores_nulos(dataframe, columnas):
    dataframe[columnas] = dataframe[columnas].fillna('Información no disponible')
    return dataframe


#Creamos una función para calcular la media y usarla para reemplazar los valores nulos o faltantes

def reemplazar_valores_nulos_con_media(dataframe, columna):
    media = dataframe[columna].mean()
    dataframe[columna] = dataframe[columna].fillna(media)
    return dataframe

def limpiar_fecha(fecha):
    if fecha == "Información no disponible":
        return pd.NaT
    else:
        try:
            return pd.to_datetime(fecha, format="%Y-%m-%d")
        except ValueError:
            return pd.NaT
        
        

In [97]:
columnas_anidadas = ['genres','belongs_to_collection', 'production_companies', 'production_countries', 'spoken_languages']
desanidados = desanidar_columnas(df_movies1, columnas_anidadas)


In [98]:
df_movies1 = agregar_columnas_desanidadas(df_movies1, columnas_anidadas)

In [99]:
df_movies1 = reemplazar_nulos_por_cero(df_movies1, ['budget', 'revenue'])

In [100]:
#Aplicamos la función para eliminar los valores nulos de la columna 'release_date'

df_movies1 = eliminar_nulos(df_movies1, ['release_date'])


In [101]:
#Aplicamos la función creada para eliminar las columnas que no vamos a utilizar

df_movies1 = eliminar_columnas(df_movies1, ['video', 'imdb_id', 'adult', 'poster_path', 'homepage', 'genres', 'belongs_to_collection', 'production_companies', 'production_countries', 'spoken_languages'])


In [102]:
# Aplicamos la función al Dataframe

df_movies1 = calcular_retorno(df_movies1)



In [103]:

df_movies1 = formatear_fechas(df_movies1, 'release_date')


In [104]:
df_movies1['release_year'] = pd.to_datetime(df_movies1['release_date']).dt.year

In [105]:

df_movies1 = formatear_columnas(df_movies1, 'budget','release_year', 'runtime', 'revenue', 'vote_count')


In [106]:
df_movies1

,budget,id,original_language,original_title,overview,popularity,release_date,revenue,runtime,status,...,title,vote_average,vote_count,genres_desanidada,belongs_to_collection_desanidada,production_companies_desanidada,production_countries_desanidada,spoken_languages_desanidada,return,release_year
0,30000000,862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033,81,Released,...,Toy Story,7.7,5415,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","{'id': 10194, 'name': 'Toy Story Collection', ...","[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",12.451801,1995
1,65000000,8844,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249,104,Released,...,Jumanji,6.9,2413,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",4.043035,1995
2,0,15602,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0,101,Released,...,Grumpier Old Men,6.5,92,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","{'id': 119050, 'name': 'Grumpy Old Men Collect...","[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",NaN,1995
3,16000000,31357,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156,127,Released,...,Waiting to Exhale,6.1,34,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",5.090760,1995
4,0,11862,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911,106,Released,...,Father of the Bride Part II,5.7,173,"[{'id': 35, 'name': 'Comedy'}]","{'id': 96871, 'name': 'Father of the Bride Col...","[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",inf,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,0,439050,fa,رگ خواب,Rising and falling between a man and woman.,0.072051,NaN,0,90,Released,...,Subdue,4.0,1,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",NaN,[],"[{'iso_3166_1': 'IR', 'name': 'Iran'}]","[{'iso_639_1': 'fa', 'name': 'فارسی'}]",NaN,0
45462,0,111109,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,2011-11-17,0,360,Released,...,Century of Birthing,9.0,3,"[{'id': 18, 'name': 'Drama'}]",NaN,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]","[{'iso_639_1': 'tl', 'name': ''}]",NaN,2011
45463,0,67758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0,90,Released,...,Betrayal,3.8,6,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",NaN,2003
45464,0,227506,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,1917-10-21,0,87,Released,...,Satan Triumphant,0.0,0,[],NaN,"[{'name': 'Yermoliev', 'id': 88753}]","[{'iso_3166_1': 'RU', 'name': 'Russia'}]",[],NaN,1917


In [107]:
# Obtener el nombre de las columnas con valores nulos
columnas_con_nulos = df_movies1.columns[df_movies1.isnull().any()].tolist()

# Imprimir el nombre de las columnas con valores nulos
for columna in columnas_con_nulos:
    print(f"La columna '{columna}' contiene valores nulos.")


La columna 'original_language' contiene valores nulos.
La columna 'overview' contiene valores nulos.
La columna 'popularity' contiene valores nulos.
La columna 'release_date' contiene valores nulos.
La columna 'status' contiene valores nulos.
La columna 'tagline' contiene valores nulos.
La columna 'title' contiene valores nulos.
La columna 'vote_average' contiene valores nulos.
La columna 'belongs_to_collection_desanidada' contiene valores nulos.
La columna 'production_companies_desanidada' contiene valores nulos.
La columna 'production_countries_desanidada' contiene valores nulos.
La columna 'spoken_languages_desanidada' contiene valores nulos.
La columna 'return' contiene valores nulos.


In [108]:
#Limpiamos los valores extraños de la columna 'popularity'

import re

df_movies1['popularity'] = df_movies1['popularity'].apply(lambda x: float(re.findall(r'\d+\.?\d*', str(x))[0]) if re.findall(r'\d+\.?\d*', str(x)) else 0)

In [109]:
#Aplicamos la función correspondiente para reemplazar los valores nulos de las columnas

df_movies1 = reemplazar_valores_nulos(df_movies1, ['original_language', 'overview', 'popularity', 'release_date', 'status', 'tagline', 'title', 'belongs_to_collection_desanidada', 'production_companies_desanidada', 'production_countries_desanidada', 'spoken_languages_desanidada', 'return'])


In [110]:
df_movies1 = reemplazar_valores_nulos_con_media(df_movies1, 'vote_average')


Leemos el otro dataset como dataframe

In [111]:
df_credits = pd.read_csv(credits)


In [ ]:
#Copiamos el dataset

df_credits = pd.read_csv(credits)
df_credits1 = pd.read_csv(credits)

In [ ]:

df_credits1['id'] = df_credits1['id'].astype(str)  # Convertimos columna 'id' a tipo 'object'

df_unidos = pd.merge(df_movies1, df_credits1, on='id', how='inner')  # Fusionamos los dataframes


In [ ]:
#Eliminamos esas cadenas de 24 caracteres de la columna 'crew'

df_unidos['crew'] = df_unidos['crew'].str.replace(r'\b\w{24}\b', '', regex=True)


In [ ]:
# Aplicamos la limpieza a la columna 'release_date'
df_unidos['release_date'] = df_unidos['release_date'].apply(limpiar_fecha)

Comenzamos con las consultas

In [ ]:
import pandas as pd
from fastapi import FastAPI

# Crear una instancia de FastAPI
app = FastAPI()

meses = {
    'enero': 1,
    'febrero': 2,
    'marzo': 3,
    'abril': 4,
    'mayo': 5,
    'junio': 6,
    'julio': 7,
    'agosto': 8,
    'septiembre': 9,
    'octubre': 10,
    'noviembre': 11,
    'diciembre': 12
}

@app.get('/cantidad_filmaciones_mes')
def cantidad_filmaciones_mes(mes: str):
    mes = mes.lower()
    
    if mes not in meses:
        return f"No se reconoce el mes '{mes.capitalize()}'"
    
    # Obtener el número del mes a partir del diccionario de mapeo
    numero_mes = meses[mes]
    
    # Filtrar las filas que corresponden al mes específico
    peliculas_mes = df_unidos[df_unidos['release_date'].dt.month == numero_mes]
    
    # Obtener la cantidad de películas estrenadas en el mes
    cantidad_peliculas = peliculas_mes.shape[0]
    
    # Devolver la cantidad de películas en la respuesta
    return f"{cantidad_peliculas} cantidad de películas fueron estrenadas en el mes de {mes.capitalize()}"

@app.get('/cantidad_filmaciones_dia')
def cantidad_filmaciones_dia(dia: str):
    dia = dia.lower()
    
    if dia not in ('lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo'):
        return f"No se reconoce el día '{dia.capitalize()}'"
    
    # Filtrar las filas que corresponden al día de la semana específico
    peliculas_dia = df_unidos[df_unidos['release_date'].apply(lambda x: obtener_dia_semana(x) == dia)]
    
    # Obtener la cantidad de películas estrenadas en el día
    cantidad_peliculas = peliculas_dia.shape[0]
    
    # Devolver la cantidad de películas en la respuesta
    return f"{cantidad_peliculas} cantidad de películas fueron estrenadas en los días {dia.capitalize()}"

@app.get('/score_titulo')
def score_titulo(titulo_de_la_filmacion: str):
    # Filtrar la fila correspondiente al título de la filmación
    pelicula = df_unidos[df_unidos['original_title'] == titulo_de_la_filmacion]
    
    if pelicula.empty:
        return f"No se encontró la filmación con título '{titulo_de_la_filmacion}'"
    
    # Obtener el título, año de estreno y score de la película
    titulo = pelicula['original_title'].iloc[0]
    anio_estreno = pelicula['release_year'].iloc[0]
    score = pelicula['popularity'].iloc[0]
    
    # Devolver la información en la respuesta
    return f"La película {titulo} fue estrenada en el año {anio_estreno} con un score/popularidad de {score}"

@app.get('/votos_titulo')
def votos_titulo(titulo_de_la_filmacion: str):
    # Filtrar las filas que corresponden al título de la filmación
    pelicula = df_unidos[df_unidos['original_title'] == titulo_de_la_filmacion]
    
    # Verificar si se cumplen las condiciones mínimas de votos (al menos 2000)
    cantidad_votos = int(pelicula['vote_count'].iloc[0])
    if cantidad_votos < 2000:
        return f"La película '{titulo_de_la_filmacion}' no cumple con el mínimo de 2000 votos."
    
    # Obtener el título, cantidad de votos y valor promedio de las votaciones
    titulo = pelicula['original_title'].iloc[0]
    promedio_votos = pelicula['vote_average'].iloc[0]
    
    # Devolver la información en la respuesta
    return f"La película '{titulo}' cuenta con un total de {cantidad_votos} valoraciones, con un promedio de {promedio_votos}."

@app.get('/get_actor')
def get_actor(nombre_actor: str):
    # Filtrar las filas que corresponden al nombre del actor
    peliculas_actor = df_unidos[df_unidos['cast'].str.contains(nombre_actor, case=False, na=False)]
    
    # Filtrar las filas que corresponden a películas en las que el actor no es director
    peliculas_actor = peliculas_actor[~peliculas_actor['crew'].str.contains(nombre_actor, case=False, na=False)]
    
    # Obtener la cantidad de películas en las que ha participado el actor
    cantidad_peliculas = peliculas_actor.shape[0]
    
    # Verificar si el actor ha participado en al menos una película
    if cantidad_peliculas == 0:
        return f"El actor {nombre_actor} no ha participado en ninguna película."
    
    # Convertir los valores de la columna 'revenue' a tipo numérico
    peliculas_actor['revenue'] = pd.to_numeric(peliculas_actor['revenue'], errors='coerce')
    
    # Eliminar las filas con valores nulos en la columna 'revenue'
    peliculas_actor = peliculas_actor.dropna(subset=['revenue'])
    
    # Calcular el retorno total del actor sumando los retornos de las películas en las que ha participado
    retorno_total = peliculas_actor['revenue'].sum()
    
    # Calcular el promedio de retorno por película
    promedio_retorno = retorno_total / cantidad_peliculas
    
    # Devolver la información en la respuesta
    return f"El actor {nombre_actor} ha participado en {cantidad_peliculas} filmaciones. Ha conseguido un retorno total de {retorno_total} con un promedio de {promedio_retorno} por filmación."

@app.get("/get_director/{nombre}", tags=['Consulta 6'])
def nombre_director(nombre: str):
    # Filtrar las filas en las que el director aparece en la columna "crew_name" y "crew_job" contiene "Director"
    director_movies = df_unidos[(df_unidos['crew'].str.contains(nombre, case=False)) & (df_unidos['crew_job'] == "Director")]

    # Verificar si se encontraron películas del director
    if director_movies.empty:
        return {"mensaje": f"No se encontró al director {nombre} en la base de datos."}

    # Obtener los ID de las películas en las que el director ha trabajado
    movie_ids = director_movies['id'].tolist()

    # Filtrar el DataFrame "df_unidos" para obtener los nombres, años, presupuestos, ingresos y relación de las películas correspondientes
    movies = df_unidos[df_unidos['id'].isin(movie_ids)]

    # Calcular las ganancias sumando todas las relaciones de las películas
    ganancias = round(movies['return'].sum(), 2)

    # Crear una lista de diccionarios con los ID, nombres, años, presupuestos, ingresos y relación de las películas
    movie_info = []
    for _, row in movies.iterrows():
        movie_info.append({
            "id": row['id'],
            "titulo": row['original_title'],
            "anio": row['release_year'],
            "presupuesto": row['budget'],
            "ingresos": row['revenue'],
            "relacion": row['return']
        })

    return {
        "nombre_director": nombre,
        "ganancias": ganancias,
        "peliculas": movie_info
    }



EDA

In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import numpy as np

